<a href="https://colab.research.google.com/github/Stergios-Konstantinidis/DMML2022_Nestle/blob/main/DMML2022_Nestle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining and Machine Learning - Project

## Detecting Difficulty Level of French Texts

### Step by step guidelines

The following are a set of step by step guidelines to help you get started with your project for the Data Mining and Machine Learning class. 
To test what you learned in the class, we will hold a competition. You will create a classifier that predicts how the level of some text in French (A1,..., C2). The team with the highest rank will get some goodies in the last class (some souvenirs from tech companies: Amazon, LinkedIn, etc).

**2 people per team**

Choose a team here:
https://moodle.unil.ch/mod/choicegroup/view.php?id=1305831


#### 1. 📂 Create a public GitHub repository for your team using this naming convention `DMML2022_[your_team_name]` with the following structure:
- data (folder) 
- code (folder) 
- documentation (folder)
- a readme file (.md): *mention team name, participants, brief description of the project, approach, summary of results table and link to the explainatory video (see below).*

All team members should contribute to the GitHub repository.

#### 2. 🇰 Join the competititon on Kaggle using the invitation link we sent on Slack.

Under the Team tab, save your team name (`UNIL_your_team_name`) and make sure your team members join in as well. You can merge your user account with your teammates in order to create a team.

#### 3. 📓 Read the data into your colab notebook. There should be one code notebook per team, but all team members can participate and contribute code. 

You can use either direct the Kaggle API and your Kaggle credentials (as explained below and **entirely optional**), or dowload the data form Kaggle and upload it onto your team's GitHub repository under the data subfolder.

#### 4. 💎 Train your models and upload the code under your team's GitHub repo. Set the `random_state=0`.
- baseline
- logistic regression with TFidf vectoriser (simple, no data cleaning)
- KNN & hyperparameter optimisation (simple, no data cleaning)
- Decision Tree classifier & hyperparameter optimisation (simple, no data cleaning)
- Random Forests classifier (simple, no data cleaning)
- another technique or combination of techniques of your choice

BE CREATIVE! You can use whatever method you want, in order to climb the leaderboard. The only rule is that it must be your own work. Given that, you can use all the online resources you want. 

#### 5. 🎥 Create a YouTube video (10-15 minutes) of your solution and embed it in your notebook. Explain the algorithms used and the evaluation of your solutions. *Select* projects will also be presented live by the group during the last class.


### Submission details (one per team)

1. Download a ZIPped file of your team's repository and submit it in Moodle here. IMPORTANT: in the comment of the submission, insert a link to the repository on Github.
https://moodle.unil.ch/mod/assign/view.php?id=1305833



### Grading (one per team)
- 20% Kaggle Rank
- 50% code quality (using classes, splitting into proper files, documentation, etc)
- 15% github quality (include link to video, table with progress over time, organization of code, images, etc)
- 15% video quality (good sound, good slides, interesting presentation).

## Some further details for points 3 and 4 above.

### 3. Read data into your notebook with the Kaggle API (optional but useful). 

You can also download the data from Kaggle and put it in your team's repo the data folder.

In [7]:
# reading in the data via the Kaggle API

# mount your Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
# install Kaggle
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### IMPORTANT
Log into your Kaggle account, go to Account > API > Create new API token. You will obtain a kaggle.json file. Save it in your Google Drive (not in a folder, in your general drive).

In [9]:
!mkdir ~/.kaggle

In [10]:
#read in your Kaggle credentials from Google Drive
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json


In [11]:
!mkdir data


In [12]:
# download the dataset from the competition page
try:
  df = pd.read_csv('/content/data/training_data.csv')
except:
  !kaggle competitions download -c detecting-french-texts-difficulty-level-2022
  !unzip "detecting-french-texts-difficulty-level-2022.zip" -d data

  0% 0.00/303k [00:00<?, ?B/s]
100% 303k/303k [00:00<00:00, 24.6MB/s]
Archive:  detecting-french-texts-difficulty-level-2022.zip
  inflating: data/sample_submission.csv  
  inflating: data/training_data.csv  
  inflating: data/unlabelled_test_data.csv  


In [13]:
# read in your training data
import pandas as pd
import numpy as np
import sklearn 
import sklearn.model_selection

df = pd.read_csv('/content/data/training_data.csv')

In [15]:
df.head()

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1


Have a look at the data on which to make predictions.

In [14]:
df_pred = pd.read_csv('/content/data/unlabelled_test_data.csv')
df_pred.head()

,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,2,"Et, paradoxalement, boire froid n'est pas la b..."
3,3,"Ce n'est pas étonnant, car c'est une saison my..."
4,4,"Le corps de Golo lui-même, d'une essence aussi..."


And this is the format for your submissions.

In [ ]:
df_example_submission = pd.read_csv('/content/data/sample_submission.csv')
df_example_submission.head()

### 4. Train your models

Set your X and y variables. 
Set the `random_state=0`
Split the data into a train and test set using the following parameters `train_test_split(X, y, test_size=0.2, random_state=0)`.

#### 4.1.Baseline
What is the baseline for this classification problem?

In [17]:
#IMPORTS

import spacy
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
import spacy.cli

import seaborn as sns


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
np.random.seed = 0

In [95]:
#Split data set

X= df['sentence']
y= df['difficulty']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=0)


#### 4.2. Logistic Regression (without data cleaning)

Train a simple logistic regression model using a Tfidf vectoriser.

In [124]:
texts = df['sentence']
tfidf = TfidfVectorizer(ngram_range=(1, 1))
features = tfidf.fit_transform(texts)
pd.DataFrame(
    features.todense(),
    columns=tfidf.get_feature_names())

,000,02h00,03h00,10,100,1000,10000,105,11,110,...,événement,événements,êtes,être,êtres,êut,île,îles,ôta,ôter
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.200821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [125]:
text_transformer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True, max_features=150000)
X_train_text = text_transformer.fit_transform(X_train)
X_test_text = text_transformer.transform(X_test)




Calculate accuracy, precision, recall and F1 score on the test set.

In [ ]:
# train accuracy with CV


LR = LogisticRegressionCV(solver='lbfgs', cv=16, max_iter=1000, random_state = 50)

LR.fit(X_train_text, y_train)

LR.score(X_train_text, y_train)

accur_train_LR = LR.score(X_train_text, y_train)


In [ ]:
#Overfitting for sure
accur_train_LR


In [ ]:
accur_test_LR = LR.score(X_test_text, y_test)
accur_test_LR

In [ ]:
y_pred = LR.predict(X_test_text)
precision_score_LR_test =  "Precision Score : ",precision_score(y_test, y_pred, 
                                           pos_label='positive',
                                           average='micro')
recall_score_LR_test = "Recall Score : ",recall_score(y_test, y_pred, 
                                           pos_label='positive',
                                           average='micro')
print(precision_score_LR_test)
print(recall_score_LR_test)

In [ ]:
def evaluate(test, pred):
  f1= f1_score(test, pred,average='micro')
  print(f'CONFUSION MATRIX:\n{confusion_matrix(test, pred )}')
evaluate(y_test, y_pred)


In [ ]:
i=0
for text in X_test_text:
  print((X_test.reset_index())._get_value(i, "sentence", takeable=False) + "  " + LR.predict(text))
  i+=1

In [ ]:
df_pred = pd.read_csv('/content/data/unlabelled_test_data.csv')
df_pred_text = text_transformer.transform(df_pred["sentence"])

df_pred['difficulty'] = list(map(lambda x : LR.predict(x)[0], df_pred_text))

df_pred = df_pred[["id", "difficulty"]]
df_pred = df_pred.set_index('id')

df_pred.to_csv('file_name.csv')
df_pred.head()


Have a look at the confusion matrix and identify a few examples of sentences that are not well classified.

Generate your first predictions on the `unlabelled_test_data.csv`. make sure your predictions match the format of the `unlabelled_test_data.csv`.

#### 4.3. KNN (without data cleaning)

Train a KNN classification model using a Tfidf vectoriser. Show the accuracy, precision, recall and F1 score on the test set.

In [100]:
X_train_text = text_transformer.fit_transform(X_train)
X_test_text = text_transformer.transform(X_test)

Try to improve it by tuning the hyper parameters (`n_neighbors`,   `p`, `weights`).

In [104]:
#KNN with 6 neightbors accuracy 0.2073
#KNN with 8 Neightbors accuracy 0.2073

knn = KNeighborsClassifier()
#K in range from 1 to 8
k_range = list(range(1, 20))
param_grid = dict(n_neighbors=k_range)

# defining parameter range
knn_cv = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1)
  
# fitting the model for grid search
knn_cv.fit(X_train_text, y_train)

y_pred_knn = knn_cv.predict(X_test_text)


Fitting 10 folds for each of 19 candidates, totalling 190 fits


In [ ]:
#KNN with minkowski and algorithm brute give us accuracy of: 0.1719
KNeighborsClassifier(algorithm='brute', leaf_size=10, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')
knn.fit(X_train_text, y_train)

y_pred_knn = knn.predict(X_test_text)

In [ ]:
#avec micro partout la meme valeur 0.2073
#avec macro Precision , recall et Score different
def evaluate(test, pred):
  precision = precision_score(test, pred, pos_label='positive',
                                           average='macro')
  recall = recall_score(test, pred, pos_label='positive',
                                           average='macro')
  f1= f1_score(test, pred, pos_label='positive',
                                           average='macro')
  print(f"ACCURACY SCORE:\n{accuracy_score(test, pred) :.4f}")
  print(f'CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}')

evaluate(y_test, y_pred_knn)

#### 4.4. Decision Tree Classifier (without data cleaning)

Train a Decison Tree classifier, using a Tfidf vectoriser. Show the accuracy, precision, recall and F1 score on the test set.

Try to improve it by tuning the hyper parameters (`max_depth`, the depth of the decision tree).

In [ ]:
# first we tried depth 10, and accuracy of 0.1885
# with deph=16 we have accuracy =0.1969
# with deph=26 accuracy = 0.2042
# wuth deph =40. accuracy=0.2021

tree = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=4000, 
                       random_state=50)
tree.fit(X_train_text, y_train)
y_pred_tree = tree.predict(X_test_text)

In [ ]:
def evaluate(train, pred):
  precision = precision_score(train, pred,pos_label='positive',
                                           average='micro')
  recall = recall_score(train, pred,pos_label='positive',
                                           average='micro')
  f1= f1_score(train, pred,pos_label='positive',
                                           average='micro')
  print(f"ACCURACY SCORE:\n{accuracy_score(train, pred) :.4f}")
  print(f'CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}')
evaluate(y_test, y_pred_tree)

ACCURACY SCORE:
0.3021
CLASSIFICATION REPORT:
	Precision: 0.3021
	Recall: 0.3021
	F1_Score: 0.3021


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


#### 4.5. Random Forest Classifier (without data cleaning)

Try a Random Forest Classifier, using a Tfidf vectoriser. Show the accuracy, precision, recall and F1 score on the test set.

In [128]:
#One hot encoder ou label encoder 

df['oe_difficulty'] = ['0' if x == 'A1'
                   else '1' if x =='A2'
                   else '2' if x == 'B1'
                   else '3' if x=='B2'
                   else '4' if x== 'C1'
                   else '5'
                   for x in df.difficulty]

df.oe_difficulty.value_counts()
newY = df['oe_difficulty']

X= df['sentence']
newY = df['oe_difficulty']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, newY, test_size=0.2, random_state=0)

In [ ]:
texts = df['sentence']
tfidf = TfidfVectorizer(ngram_range=(1, 1))
features = tfidf.fit_transform(texts)
pd.DataFrame(
    features.todense(),
    columns=tfidf.get_feature_names())

In [130]:
text_transformer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True, max_features=150000)
X_train_text = text_transformer.fit_transform(X_train)
X_test_text = text_transformer.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train_text, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train_text, y_train)
base_accuracy = evaluate(base_model, X_test_text, y_test)


best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test_text, y_test)


print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))


In [88]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search = GridSearchCV(estimator = RandomForestRegressor(), param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train_text, y_train)
grid_search.best_params_
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test_text, y_test)
print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

NameError: ignored

In [85]:
#X, y = make_classification(n_samples=1000, n_features=4,
                          #n_informative=2, n_redundant=0,
                          #random_state=0, shuffle=False)


clf = RandomForestClassifier(random_state=0, bootstrap= True, max_depth= 80,
 max_features= 3, min_samples_leaf =5, n_estimators= 100)

clf.fit(X_train_text, y_train)
RandomForestClassifier(...)

y_pred_forest =  clf.predict(X_test_text)




NameError: ignored

In [81]:
def evaluate(train, pred):
  precision = precision_score(train, pred,pos_label='positive',
                                           average='micro')
  recall = recall_score(train, pred,pos_label='positive',
                                           average='micro')
  f1= f1_score(train, pred,pos_label='positive',
                                           average='micro')
  print(f'CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}')
evaluate(y_test, y_pred_forest)

CLASSIFICATION REPORT:
	Precision: 0.1646
	Recall: 0.1646
	F1_Score: 0.1646


In [ ]:
clf.score(X_train_text, y_train)
accur_train_forest = clf.score(X_train_text, y_train)
accur_train_forest

In [ ]:
clf.score(X_test_text, y_pred)
accur_test_forest = clf.score(X_test_text, y_test)
accur_test_forest

#### 4.6. Any other technique, including data cleaning if necessary

Try to improve accuracy by training a better model using the techniques seen in class, or combinations of them.

As usual, show the accuracy, precision, recall and f1 score on the test set.

In [ ]:
df.head()

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1


In [ ]:
#first let's check if there are NA's
df.isnull().sum()

id            0
sentence      0
difficulty    0
dtype: int64

In [ ]:
le_diff = pd.Series(LabelEncoder().fit_transform(df["difficulty"]), name="le_diff")
le_diff

0       4
1       0
2       0
3       0
4       2
       ..
4795    3
4796    4
4797    1
4798    5
4799    5
Name: le_diff, Length: 4800, dtype: int64

#### 4.7. Show a summary of your results

In [ ]:
tableau_data = {'Logistic Regression': [0.565376, accur_test_LR, accur_train_LR,  precision_score_LR_train, recall_score_LR_train],
        'KNN': [0.565376, accur_test_KNN , accur_train_KNN, precision_score_KNN, recall_score_KNN],
        'Decision Tree' : [accur_test_tree, accur_train_tree, precision_score_tree, recall_score_tree]
        'Random Forest': [accur_test_forest, accur_train_forest, precision_score_forest, recall_score_forest]}
  
# Creates pandas DataFrame.
tableau_df = pd.DataFrame(tableau_data, index=[ 'Base rate', 'Accuracy Test',
                               'Accuracy Train',
                               'Precision',
                               'Recall'])

tableau_df

In [ ]:
#First we use an ordinal encoder

df['oe_difficulty'] = ['0' if x == 'A1'
                   else '1' if x =='A2'
                   else '1' if x== 'B1'
                   else '3' if x == 'B2'
                   else '4' if x == 'C1'
                   else '5' 
                   for x in df.difficulty]   

In [ ]:
#Instantiate the encoder
oe=OrdinalEncoder()

# set the order of your categories
oe.set_params(categories= [['0', '1', '2', '3', '4','5']])

# fit-transform a dataframe of the categorical age variable
oe_difficulty =oe.fit_transform(df[['oe_difficulty']])

#number of values per class
oe_difficulty = pd.DataFrame(oe_difficulty).astype('int')
oe_difficulty.value_counts()

1    1590
0     813
5     807
4     798
3     792
dtype: int64

In [ ]:
#drop column [difficulty] which is not encoded
df.drop('difficulty', axis =1)

,id,sentence,oe_difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,4
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",0
2,2,Le test de niveau en français est sur le site ...,0
3,3,Est-ce que ton mari est aussi de Boston?,0
4,4,"Dans les écoles de commerce, dans les couloirs...",1
...,...,...,...
4795,4795,"C'est pourquoi, il décida de remplacer les hab...",3
4796,4796,Il avait une de ces pâleurs splendides qui don...,4
4797,4797,"Et le premier samedi de chaque mois, venez ren...",1
4798,4798,Les coûts liés à la journalisation n'étant pas...,5


In [ ]:
#Overfitting with cross validation = 5, accuracy =1
#Cross validation =10, accuracy = 0.9989

LR_cv = LogisticRegressionCV(solver='lbfgs', cv=10, max_iter=100, random_state = 0)

LR_cv.fit(X_train_text, y_train)

LR_cv.score(X_train_text, y_train)

accur_train_LR = LR_cv.score(X_train_text, y_train)

In [ ]:
y_pred_LR_test = LR_cv.predict(X_test_text)
accur_test_LR = LR_cv.score(X_test_text, y_pred)
accur_test_LR

In [19]:
#Arme fatale
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel

#To remove when we test the code
import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

import os
import pandas as pd
import numpy as np

# TQDM to Show Progress Bars #
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook

# SKLearn libraries for splitting sample and validation
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Additional Libraries that we are using only in this notebook
import torch
import gc

In [22]:
# Change to Working Directory with Training Data # 
#os.chdir("/content/drive/MyDrive/Colab Notebooks/SDM")

# Load Training Data #
#TrainingData = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SDM/training_data_SDM.csv", sep='\t')

# Store Data in Lists for Text Classification #
Website = np.array(X_train.values.tolist())
Website_Text = X_train.values.tolist()
Classes = y_train.values.tolist()

#Our classfiers

CLASSIFIERS = [
               ["BERT", "bert", "bert-base-uncased"],
               ["RoBERTa","roberta","roberta-base"],
               ["XLNet", "xlnet","xlnet-base-cased"],
               ["ELECTRA", "electra","google/electra-base-discriminator"],
               ["Big-Bird", "bigbird","google/bigbird-roberta-base"],
               ]
# Cross Validation #
cv = 5

# Define whether you want to manually reweight the sample by oversampling the smaller class 
Reweight = False
# Define arrays in which to store classification outputs # 
RESULTS = []
Classified_Values =[]

In [31]:
cv= 5

# Define whether you want to manually reweight the sample by oversampling the smaller class 
Reweight = False

# Define arrays in which to store classification outputs # 
RESULTS = []
Classified_Values =[]

# Loop Through Different Classifiers #
for CL in tqdm_notebook(CLASSIFIERS, desc = "Evaluating Classifiers", leave = True):

    # Extract Classifier Names & Model #
    name  = CL[0]
    Model1 = CL[1]
    Model2 = CL[2]
    
    # Define Arrays to store Actual, Predicted and Ids variables (Because we are shuffling them in next step) # 
    y_actual = []
    y_predicted = []
    web_s = []

    # Loop through K Folds and Repeat Cross Validation #
        
    KFoldSplitter = StratifiedKFold(n_splits = cv, shuffle = True, random_state = 1)
        
    for train_i, test_i in tqdm_notebook(KFoldSplitter.split(Website_Text, Classes), 
                                            desc = 'Cross-Validating',
                                            leave = False,
                                            total = cv):
      
        # Select Rows in Data Based on Indexes [train_i, test_i]
        Y = np.array(Classes)

        Website_Text_Array = np.array(Website_Text)

        train_X, test_X = Website_Text_Array[train_i], Website_Text_Array[test_i]
        train_y, test_y = Y[train_i], Y[test_i]
        Train_Website, Test_Website = Website[train_i], Website[test_i]

        # Create Training Data in Paired Format (Nessesary for Transformers) # 
        TrainingDataframe = list(zip( list(train_X), list(train_y)))
        TestDataframe = list(zip( list(test_X), list(test_y)))

        train_df = pd.DataFrame(TrainingDataframe)
        train_df.columns = ["text", "labels"]

        # Create a Classification Model
        model = ClassificationModel(Model1, Model2,                                   
                                    args={'num_train_epochs':5,
                                          'overwrite_output_dir': True,
                                          'use_early_stopping':False,
                                          'train_batch_size':50,
                                          'do_lower_case':True, 
                                          'silent':True,
                                          'no_cache':True, 
                                          'no_save':True}
                                    )                       
                                                  

    # Train the Model
    model.train_model(train_df)

    # Predict on Holdout Sample #
    predictions, raw_outputs = model.predict( list(test_X) )

    # Store Output #
    web_s = web_s + list(Test_Website)
    y_actual = y_actual + list(test_y)
    y_predicted = y_predicted + list(predictions)

    gc.collect()
    

    # ---------------------------------------------------------- #
    # This runs only after all of the folds have been classified # 
    # ---------------------------------------------------------- #

    # Compute the Share of AI Patents #
    Share = np.round(np.mean(y_predicted), 3)

    # Calculate Model Performance Metrics #
    Accuracy = accuracy_score(y_actual, y_predicted)
    ROC = roc_auc_score(y_actual, y_predicted)
    Precision = precision_score(y_actual, y_predicted)
    Recall = recall_score(y_actual, y_predicted)
    F1 = f1_score(y_actual, y_predicted)
    CM = confusion_matrix(y_actual, y_predicted)

    # Round to 3 Decimal Places # 
    #FN = np.round(CM[0][0]/CM[0].sum(), 3)
    #FP = np.round(CM[0][1]/CM[0].sum(), 3)
    #TN = np.round(CM[1][0]/CM[1].sum(), 3)
    #TP = np.round(CM[1][1]/CM[1].sum(), 3)

    FN = np.round(CM[0][0]/(CM[0][0] + CM[1][0]), 3)
    FP = np.round(CM[0][1]/(CM[0][1] + CM[1][1]), 3)
    TN = np.round(CM[1][0]/(CM[0][0] + CM[1][0]), 3)
    TP = np.round(CM[1][1]/(CM[0][1] + CM[1][1]), 3)
    
    # Add Classification Performance Metrics to List#
    RESULTS.append([name, Share, TP, FN, FP, TN,
                                            np.round(Accuracy, 3),
                                            np.round(ROC, 3),
                                            np.round(Precision, 3),
                                            np.round(Recall, 3),
                                            np.round(F1, 3)])


    # Add Classification Results to List # 
    Classified_Values.append(list(zip(len(web_s)*[name],web_s, y_actual, y_predicted)))




Evaluating Classifiers:   0%|          | 0/5 [00:00<?, ?it/s]

Cross-Validating:   0%|          | 0/5 [00:00<?, ?it/s]

ValueError: ignored

In [ ]:
# Convert List to Dataframe #
RESULTS_TABLE = pd.DataFrame(RESULTS, columns = ["Name", "Share", "True-Positives", 
                                                 "False-Negatives", "False-Positives", 
                                                 "True-Negatives","Accuracy", "AUC", 
                                                 "Precision", "Recall", "F1"] )

RESULTS_TABLE["Type"] = "Transformer"
RESULTS_TABLE = RESULTS_TABLE[["Name", "Type", "Share", "True-Positives", 
                               "False-Negatives", "False-Positives", 
                               "True-Negatives","Accuracy", "AUC", 
                               "Precision", "Recall", "F1"]]



# Output Results #
RESULTS_TABLE.sort_values("Accuracy", ascending = False ).to_csv("/content/drive/MyDrive/Colab Notebooks/SDM/Output/Model Performance/Transformer Classification Model Performance.csv")

# Display Results -- Out of Sample (Holdout) prediction -- Sorted by Accuracy #
RESULTS_TABLE.sort_values("Accuracy", ascending = False )

In [ ]:
text_transformer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True, max_features=150000)
X_fit_transform_text = text_transformer.fit_transform(X)
X_transform_text = text_transformer.transform(X)

# train accuracy with CV
cvt=8

while cvt < 19:
  LR_CV_FT = 0
  LR_CV_FT = 0


  LR_CV_FT = LogisticRegressionCV(solver='lbfgs', cv=cvt, max_iter=100, random_state = 50)
  LR_CV_FT.fit(X_fit_transform_text, y)
  df_pred = pd.read_csv('/content/data/unlabelled_test_data.csv')
  df_pred_text = text_transformer.transform(df_pred["sentence"])
  df_pred['difficulty'] = list(map(lambda x : LR_CV_FT.predict(x)[0], df_pred_text))
  df_pred = df_pred[["id", "difficulty"]]
  df_pred = df_pred.set_index('id')
  df_pred.to_csv('file_name.csv')
  df_pred.head()
  !kaggle competitions submit -c detecting-french-texts-difficulty-level-2022 -f file_name.csv -m str("LogisticRegressionCV(solver='lbfgs', cv="cvt", max_iter=1000, random_state = 50) text Fit Tranform")

  LR_CV_T = LogisticRegressionCV(solver='lbfgs', cv=cvt, max_iter=100, random_state = 50)
  LR_CV_T.fit(X_transform_text, y)
  df_pred = pd.read_csv('/content/data/unlabelled_test_data.csv')
  df_pred_text = text_transformer.transform(df_pred["sentence"])
  df_pred['difficulty'] = list(map(lambda x : LR_CV_T.predict(x)[0], df_pred_text))
  df_pred = df_pred[["id", "difficulty"]]
  df_pred = df_pred.set_index('id')
  df_pred.to_csv('file_name.csv')
  df_pred.head()
  !kaggle competitions submit -c detecting-french-texts-difficulty-level-2022 -f file_name.csv -m str("LogisticRegressionCV(solver='lbfgs', cv="cvt", max_iter=1000, random_state = 50) text Tranform")
  cvt+=2






/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `kaggle competitions submit -c detecting-french-texts-difficulty-level-2022 -f file_name.csv -m str("LogisticRegressionCV(solver='lbfgs', cv="cvt", max_iter=1000, random_state = 50) text Fit Tranform")'


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


KeyboardInterrupt: ignored

In [ ]:
x=0

if x==1:
  df_pred = pd.read_csv('/content/data/unlabelled_test_data.csv')
  df_pred_text = text_transformer.transform(df_pred["sentence"])
  df_pred['difficulty'] = list(map(lambda x : LR.predict(x)[0], df_pred_text))
  df_pred = df_pred[["id", "difficulty"]]
  df_pred = df_pred.set_index('id')
  df_pred.to_csv('file_name.csv')
  df_pred.head()
  !kaggle competitions submit -c detecting-french-texts-difficulty-level-2022 -f file_name.csv -m "LR"

100% 8.30k/8.30k [00:01<00:00, 7.75kB/s]
Successfully submitted to Detecting the difficulty level of French texts